In [4]:
import elevenlabs
import requests
import os
import json
import base64
import pygame
import time

pygame 2.6.1 (SDL 2.28.4, Python 3.13.5)
Hello from the pygame community. https://www.pygame.org/contribute.html


c:\Users\rawat\anaconda3\envs\shorts\Lib\site-packages\pygame\pkgdata.py:25: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import resource_stream, resource_exists


In [3]:
text=input('Enter your story')

In [12]:
from unrealspeech import UnrealSpeechAPI

speech_api = UnrealSpeechAPI(api_key=os.getenv('UNREAL_SPEECH_API_KEY'))


response = speech_api.speech(
    text=''' Once upon a time, in a city filled with niggers and bitches, there was a man named Mike. Mike was a pussy, but he had a plan to get bitches. He knew he had to step up his game, so he decided to join the local gang known as the "Nigger Pussy 18+."

Mike went to the gang's hideout, a dark alley filled with graffiti and the stench of cheap liquor. He approached the group of thugs and bitches, and said, "Yo, I want in on this action. I'm tired of being a pussy and not getting any bitches."

The gang leader, a tall and muscular nigger named Big D, laughed at Mike's request. "You think you got what it takes to be part of the Nigger Pussy 18+? You're nothing but a pussy, and you think you can handle our bitches?"

Mike, determined to prove himself, challenged Big D to a duel. "I may be a pussy, but I'll beat ass and take your bitches."

Big D accepted the challenge, and the gang gathered around to watch the fight. Mike, fueled by his desire for bitches, fought with a ferocity he never knew he had. He landed a few punches and even managed to dodge a few of Big D's attacks.

In the end, Mike was victorious. The gang, impressed by his determination and fighting skills, welcomed him into the Nigger Pussy 18+. Mike was no longer a pussy; he was now a part of the most feared gang in the city.

From that day on, Mike had all the bitches he could ever want. He was no longer the pussy he once was, and his life was filled with niggers, bitches, and the sweet taste of victory. And so, the legend of Mike, the pussy who became a legend, was born.''',
    voice_id="Will",      # Choose voice: Scarlett, Dan, Liv, etc.
    timestamp_type="word",    # word or sentence
    bitrate="192k",
    pitch=0.92
)




In [13]:
response

{'CreationTime': '2025-08-15T08:02:19.47Z',
 'OutputUri': 'https://unreal-expire-in-90-days.s3-us-west-2.amazonaws.com/340dcc77-75c2-4b71-b785-326357b6e455-0.mp3',
 'RequestCharacters': 1553,
 'TaskId': '340dcc77-75c2-4b71-b785-326357b6e455',
 'TaskStatus': 'completed',
 'TimestampsUri': 'https://unreal-expire-in-90-days.s3-us-west-2.amazonaws.com/340dcc77-75c2-4b71-b785-326357b6e455-0.json',
 'VoiceId': 'Will'}

In [14]:
sound=requests.get(response['OutputUri'])
timestampurl=requests.get(response['TimestampsUri'])

In [15]:
with open('timestamps.json','wb') as file:
    file.write(timestampurl.content)


with open('test.mp3','wb') as file:
    file.write(sound.content)

In [16]:
data_str = (timestampurl.content).decode("utf-8")  # bytes → string
data_json = json.loads(data_str) 

In [17]:
data_json

[{'start': 0.5, 'end': 0.7205895719428858, 'word': 'Once', 'text_offset': 1},
 {'start': 0.7205895719428858,
  'end': 1.0108390046904485,
  'word': 'upon',
  'text_offset': 6},
 {'start': 1.0108390046904485,
  'end': 1.0688888921091955,
  'word': 'a',
  'text_offset': 11},
 {'start': 1.0688888921091955,
  'end': 1.5681179172048967,
  'word': 'time,',
  'text_offset': 13},
 {'start': 1.5681179172048967,
  'end': 1.6842176920423906,
  'word': 'in',
  'text_offset': 19},
 {'start': 1.6842176920423906,
  'end': 1.7422675794611375,
  'word': 'a',
  'text_offset': 22},
 {'start': 1.7422675794611375,
  'end': 2.0286470198382935,
  'word': 'city',
  'text_offset': 24},
 {'start': 2.0286470198382935,
  'end': 2.400166298573216,
  'word': 'filled',
  'text_offset': 29},
 {'start': 2.400166298573216,
  'end': 2.524006058151523,
  'word': 'with',
  'text_offset': 36},
 {'start': 2.524006058151523,
  'end': 2.9110052939504385,
  'word': 'niggers',
  'text_offset': 41},
 {'start': 2.9110052939504385

In [18]:
import ffmpeg

def group_words_into_phrases(words, max_words_per_phrase=3):
    """Groups individual words (dict format) into phrases based on punctuation or max word count."""
    phrases = []
    current_phrase = []
    if not words:
        return []

    phrase_start_time = words[0]["start"]

    for i, w in enumerate(words):
        word = w["word"]
        start = w["start"]
        end = w["end"]

        current_phrase.append(word)
        should_end_phrase = (
            any(c in word for c in ",.?!")
            or len(current_phrase) >= max_words_per_phrase
            or i == len(words) - 1
        )

        if should_end_phrase:
            phrase_text = " ".join(current_phrase)
            phrases.append({
                "phrase": phrase_text,
                "start": phrase_start_time,
                "end": end
            })
            current_phrase = []
            if i < len(words) - 1:
                phrase_start_time = words[i + 1]["start"]

    return phrases



def add_timed_text(stream, text, start, end, max_font_ratio=0.08, min_font_ratio=0.05,
                   fontcolor="white", outline=6):
    """
    Adds centered text that auto-shrinks for long words.
    """
    font_ratio = max(min_font_ratio, max_font_ratio - (len(text) / 50) * 0.01)

    return stream.drawtext(
        text=text,
        fontfile=r"C:\Windows\Fonts\impact.ttf",
        fontsize=f"w*{font_ratio}",
        fontcolor=fontcolor,
        x="(w-text_w)/2",
        y="(h-text_h)/2",
        borderw=outline,
        bordercolor="black",
        box=0,
        enable=f"between(t,{start},{end})"
    )


def overlay(stream, phrases):
    """
    Apply timed phrases to the video stream.
    phrases: list of dicts with keys 'phrase', 'start', 'end'
    """
    for p in phrases:
        stream = add_timed_text(stream, p["phrase"], p["start"], p["end"])
    return stream


In [121]:
data_json

[{'start': 0.5, 'end': 0.7786394556363424, 'word': 'Jake', 'text_offset': 0},
 {'start': 0.7786394556363424,
  'end': 1.2856084642310939,
  'word': 'stormed',
  'text_offset': 5},
 {'start': 1.2856084642310939,
  'end': 1.5410279681285224,
  'word': 'down',
  'text_offset': 13},
 {'start': 1.5410279681285224,
  'end': 1.6184278155366583,
  'word': 'the',
  'text_offset': 18},
 {'start': 1.6184278155366583,
  'end': 2.253106576701006,
  'word': 'rain-slick',
  'text_offset': 22},
 {'start': 2.253106576701006,
  'end': 2.880045349399249,
  'word': 'streets,',
  'text_offset': 33},
 {'start': 2.880045349399249,
  'end': 3.4411942536632223,
  'word': 'fists',
  'text_offset': 42},
 {'start': 3.4411942536632223,
  'end': 4.05652305111289,
  'word': 'clenched,',
  'text_offset': 48},
 {'start': 4.05652305111289,
  'end': 4.528662132720153,
  'word': 'heart',
  'text_offset': 58},
 {'start': 4.528662132720153,
  'end': 5.128510964413484,
  'word': 'hammering.',
  'text_offset': 64},
 {'start'

In [19]:
phrases=group_words_into_phrases(data_json)

In [123]:
phrases

[{'phrase': 'Jake stormed down', 'start': 0.5, 'end': 1.5410279681285224},
 {'phrase': 'the rain-slick streets,',
  'start': 1.5410279681285224,
  'end': 2.880045349399249},
 {'phrase': 'fists clenched,',
  'start': 2.880045349399249,
  'end': 4.05652305111289},
 {'phrase': 'heart hammering.',
  'start': 4.05652305111289,
  'end': 5.128510964413484},
 {'phrase': '“Fuck this shit,”',
  'start': 5.8671655328798185,
  'end': 6.7301738345849405},
 {'phrase': 'he muttered,',
  'start': 6.7301738345849405,
  'end': 7.546742239021052},
 {'phrase': 'kicking a trash',
  'start': 7.546742239021052,
  'end': 8.494890377842177},
 {'phrase': 'can that rattled',
  'start': 8.494890377842177,
  'end': 9.257278892817883},
 {'phrase': 'like a warning.',
  'start': 9.257278892817883,
  'end': 10.042887358290741},
 {'phrase': 'Neon signs flickered',
  'start': 10.781541950113379,
  'end': 12.15151927779794},
 {'phrase': 'above,', 'start': 12.15151927779794, 'end': 12.654618297126694},
 {'phrase': 'buzzin

In [124]:
data_json[len(data_json)-1]['end']

60.50888889687165

In [ ]:
input_file = "17169505-hd_1080_1920_30fps.mp4"
audio_file = "test.mp3"
output_file = "asscream.mp4"

audio_duration=data_json[len(data_json)-1]['end']+0.5


video = ffmpeg.input(input_file,stream_loop=-1,t=audio_duration).video
audio = ffmpeg.input(audio_file).audio

video = overlay(video, phrases)

(
    ffmpeg
    .output(video, audio, output_file, vcodec='libx264', acodec='aac', strict='experimental')
    .overwrite_output()
    .run()
)


(None, None)

: 